# Simple Agentic Framework system
generating cold sales outreach emails (?)  
뭐하는 거지?

## Setup
- Sendgrid: https://sendgrid.com/  
Twilio의 메시지 전송 website.
- Sendgrid에서 api key 작성  
SENDGRID_API_KEY=SG.
- sender authentication에서 이메일 검증 등록

In [ ]:
# sendgrid 모듈 등록
%pip install sendgrid

In [ ]:
# 이메일 테스트 함 해봐
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [ ]:
load_dotenv(override = True)

In [ ]:
# 이메일 보내기가 될라나?
# 요거 잘 안됨. 다음 장애 나오는데 해결이 안되네.
#      URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)>
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("dinokrap@gmail.com")
    to_email = To("g1.systemswteam@gmail.com")
    content = Content("text/plain", "This is very important test email using sendgrid")
    mail = Mail(from_email, to_email, "SendGrid Email from Agents", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

## resend.com
sendgrid로 메일 보내기가 잘 되지 않음. 대안으로 resend.com으로 다시 한번 해보기.  
dinokrap으로 signin하고. 보낼 도메인을 확인하라고 한다. 나는 일단 domain은 갖고 있지 않으므로, resend의 도메인으로 한다.

In [ ]:
# pip로 resend 모듈은 설치하고
%pip install resend

In [ ]:
import os
import resend
from dotenv import load_dotenv

# 언제나 그렇듯이 환경 변수 로드
load_dotenv(override=True)

resend.api_key = os.getenv("RESEND_API_KEY")
params: resend.Emails.SendParams = {
    "from": "Acme <onboarding@resend.dev>",
    "to": "dinokrap@gmail.com",
    "subject": "Hello from visual code using resend",
    "html": "<p>This is an email sending test from vscode. <strong>Do you see it?</strong></p>"
}

email = resend.Emails.send(params)
print(email)

Resend는 메일 보내기 성공함.  
해서 Resend로 본 코드 작성 시작.  
## Simple Agentic AI Framework

In [ ]:
# 처음부터 다시
# import
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import resend
import os
import asyncio

In [ ]:
# 언제나 그렇듯이
load_dotenv(override = True)

In [ ]:
# 이메일 보내기 테스트
def send_email():
    resend.api_key = os.getenv("RESEND_API_KEY")
    params: resend.Emails.SendParams = {
    "from": "Acme <onboarding@resend.dev>",
    "to": "dinokrap@gmail.com",
    "subject": "Hello from visual code using resend",
    "html": "<p>This is an email sending test from vscode. <strong>Do you see it?</strong></p>"
    }
    email = resend.Emails.send(params)

send_email()

In [ ]:
# Agent workflow
# 각 agent에 전달할 instruction 작성하고,
instruction1 = "You are a sales agent working for ComlAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for aduits, powered by AI. \
    You write professional, serious cold emails."
instruction2 = "You are a humorous, engaing sales agent working for CompAI, \
    a company that provides a Saas tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write witty, engaging cold emails that are likely to get a response."
instruction3 = "You are busy sales agent working for ComlAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write concise, to the point cold emails."

In [ ]:
# 3개의 각 agent 인스턴스를 생성
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions=instruction1,
    model="gpt-4o-mini"
)
sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instruction2,
    model="gpt-4o-mini"
)
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instruction3,
    model="gpt-4o-mini"
)

In [ ]:
# 테스트로 agent #1을 실행하는데,
# run_streamed명령으로 생성하는 족족 결과를 실시간으로 받는다.
# 그래서 async for를 사용하고, stream_events를 실행해서 LLM이 만드는 모든 이벤트를 받는다.
result = Runner.run_streamed(sales_agent1, input="write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
# 이번에는 3개 agent를 한번에 실행하고,리턴 받는다.
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output, "\n\n")

In [ ]:
# sales picker : agent. 다른 agnet들이 쓴 이메일을 평가한다.
# instruction
instruction_sales_picker = "You pick the best cold sale email from the given options. \
    Imaginge you are a customer and pick a one you are mostly likely to respond to. \
    Do not give an explanation; reply with the selected email only."
# agent 인스턴스 생성
sales_picker_agent = Agent(
    name="sales pikcer",
    instructions=instruction_sales_picker,
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"
with trace("selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )
    outputs = [result.final_output for result in results]
    emails = "Cold sales emails: \n\n".join(outputs)
    best = await Runner.run(sales_picker_agent, emails)
    print(f"Best sales email: \n{best.final_output}")

*Simple agentic system*  
이렇게 agent들에게 cold sales email 작성을 시키고  
요 이메일들을 pick하는 agent를 생성하고, pick 시키고  
이제 실행한 trace를 살펴보라. 사이트는 아래.  
https://platform.openai.com/traces

## Use of tools
기본 agentic framework에 tool을 추가할 거야.

In [ ]:
# 먼저 agent들을 다시 생성해 보자
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instruction1,
    model="gpt-4o-mini"
)
sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instruction2,
    model="gpt-4o-mini"
)
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instruction3,
    model="gpt-4o-mini"
)

In [ ]:
# agent는 요렇게 생겼다.
sales_agent1
# Agent(
# name='Professional Sales Agent', 
# handoff_description=None, 
# tools=[], 
# mcp_servers=[], 
# mcp_config={}, 
# instructions='You are a sales agent working for ComlAI,     a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for aduits, powered by AI.     You write professional, serious cold emails.', 
# prompt=None, 
# handoffs=[], 
# model='gpt-4o-mini', 
# model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), 
# input_guardrails=[], 
# output_guardrails=[], 
# output_type=None, 
# hooks=None, 
# tool_use_behavior='run_llm_again', 
# reset_tool_choice=True)

In [ ]:
# tool : function tool
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    resend.api_key=os.getenv("RESEND_API_KEY")
    params: resend.Emails.SendParams = {
        "from": "Acme <onboarding@resend.dev>",
        "to": "dinokrap@gmail.com",
        "subject": "Sales Email",
        "html": body
    }
    email = resend.Emails.send(params)
    return {"status":"success"}

In [ ]:
# send_email은 어떻게 생겼나?
send_email
# FunctionTool(
# name='send_email', 
# description='Send out an email with the given body to all sales prospects', 
# params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, 
# on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000012A7272C7C0>, 
# strict_json_schema=True, 
# is_enabled=True)

In [ ]:
# agent를 tool로 변환
tool1 = sales_agent1.as_tool(
    tool_name="sales_agent1",
    tool_description="Write a cold sales email"
)
# 나머지 agent들도 tool로 변환
tool2 = sales_agent2.as_tool(
    tool_name="sales_agent2",
    tool_description="Write a cold sales email"
)
tool3 = sales_agent3.as_tool(
    tool_name="sales_agent3",
    tool_description="Write a cold sales email"
)

In [ ]:
# agent tool은 어떻게 생겼나? function tool하고 똑같이 생겼다.
tool1
# FunctionTool(
# name='sales_agent1', 
# description='Write a cold sales email', 
# params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, 
# on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000012A79BA6200>, 
# strict_json_schema=True, 
# is_enabled=True)

# 이제 리스트로 tool을 넣자
tools = [tool1, tool2, tool3, send_email]
tools

In [ ]:
# 이제 tool들을 사용할 agent. sales_manager agent를 만들자.
instruction_sales_manager = "You are a sales manager working for ComplAI. You use the tools given to you to write a cold sales email. \
    You never write a sales email yourself; you always use the tools. \
    You try all 3 sales_agent tools once before picking the best one. \
    You choose the single best email and use the send_email tool to send the best email (and only the best email) to the user."

sales_manager = Agent(
    name="Sales Manager",
    instructions=instruction_sales_manager,
    tools=tools,
    model="gpt-4o-mini"
)

message = "Send a cold sales email addressed to 'Dear CEO'"
with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)

다시 trace를 체크할 시간  
https://platform.openai.com/traces

## Handoffs
Handoffs 하고 Agent-as-tools는 서로 비슷. 모두 agent가 다른 agent와 같이 일하는 방법.  
다른 점은 tools는 control을 다시 받아오고 (manager agent로)  
handoffs는 control을 넘겨주고 안 받아온다.

In [ ]:
# 여기서는 email subject 쓰는 agent, txt email body를 html body로 변환하는 agent를 만들고,
# 다시 tool로 변환할 거야.

# mail subject agent
subject_instructions = "You can write a subject for a cold sales email. \
    You are given a message and you need to write a subject for an email that is likely to get a response."
subject_agent = Agent(
    name="Email Subject Writer",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)
subject_tool = subject_agent.as_tool(
    tool_name="subject_writer",
    tool_description="Write a subject for a cold sales email"
)

# html converter agent
html_instruction = "You can conver a text email body to a html email body. \
    You are given a text email body which might have some markdown \
    and you need to convert it to an HTML email body with simple, clear, compelling layout and design."
html_converter = Agent(
    name="HTML email body converter",
    instructions=html_instruction,
    model="gpt-4o-mini"
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a text email body to an HTML email body"
)

In [34]:
# function tool : 메일 보내는 function tool
@function_tool
def send_html_email(subject:str, html_body:str) -> Dict[str, str]:
    """ Send out an email with the given subject and html body to all sales prospects """
    resend.api_key=os.getenv("RESEND_API_KEY")
    params: resend.Emails.SendParams = {
        "from": "Acme <onboarding@resend.dev>",
        "to": "dinokrap@gmail.com",
        "subject": subject,
        "html": html_body
    }
    email = resend.Emails.send(params)
    return {"status":"success"}

In [35]:
tools = [subject_tool, html_tool, send_html_email]

In [36]:
# handoff agent를 정의 <- Manager agent에게서 넘겨받을거야.
emailer_instructions = "You are an email formatter and sender. You receive a body of an email to be sent. \
    You first use the subject_writer tool to write a subject for the email, then use the html_converter to convert the body to HTML. \
    Finally, you use the send_html_email tool to send the email with the subject and HTML body."
emailer_agent = Agent(
    name="Email Formatter",
    instructions=emailer_instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

In [37]:
# 자, 이제 Email Manager를 다시 정의해야지.
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
    You never generate sales email yourself; you always use the tools. \
    You try all 3 sales agent tools at least once before choosing the best one. \
    You can use the tools multiple times if you're not satisfied with the results from the first try. \
    You select single best email using your own judgement of which email will be the most effective. \
    After picking the email, you handoff to the Email Formatter agent to format and send the email."

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

이번에도 trace를 살펴봐야지.  
https://platform.openai.com/traces